In [1]:
import os

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
import torch
import math 
import copy
import random
import argparse
import torch.optim as optim



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
def load_data(dataset_name, K):
    if dataset_name == "statlog":
        ##statlog
        __context_size__ = 8
        __arm_size__ = 7
        fr = open("shuttle.trn",'r+')
        CONTEXT = []
        REWARD = []
        SYM = {}
        aa = 0
        K = 0
        for line in fr:
            K+=1
            aaa = line.split("\n")
            aaa = aaa[0].split(" ")
            temp = np.int64(aaa[9])-1
            context = np.double(aaa[1:9])
            context = context/np.linalg.norm(context)
            ttt = context
            CONTEXT.append(ttt)
            reward = np.zeros(__arm_size__)
            reward[temp] = 1
            REWARD.append(reward)

    elif dataset_name == "magic":
        ####magic gamma

        __context_size__ = 10
        __arm_size__ = 2
        fr = open("letter.data", 'r+')

        K = 0
        CONTEXT = []
        REWARD = []

        fr.close()
        fr = open("magic04.data", 'r+')
        for line in fr:

            context = []
            aaa = line.split(",")
            context = np.double(aaa[0:__context_size__])
            context = context / np.linalg.norm(context)
            K += 1
            CONTEXT.append(np.array(context))
            __context_size__ = len(context)
            reward = np.zeros(__arm_size__)
            if aaa[10] == 'g\n':
                reward[0] = 1
            else:
                reward[1] = 1
            REWARD.append(reward)

        fr.close()
    elif dataset_name == "covertype":
        ##For covertypr
        __context_size__ = 14
        __arm_size__ = 7
        fr = open("covtype.data", 'r+')
        CONTEXT = []
        REWARD = []
        SYM = {}
        aa = 0
        K = 0
        for line in fr:
            K += 1
            aaa = line.split(",")
            __context_size__ = len(aaa) - 1
            temp = int(aaa[__context_size__]) - 1
            context = np.double(aaa[0:__context_size__])
            context = context / np.linalg.norm(context)
            ttt = context
            CONTEXT.append(ttt)
            reward = np.zeros(__arm_size__)
            reward[temp] = 1
            REWARD.append(reward)
    else:
        assert('dataset not avaiable')

    TEMP_CONTEXT = []
    TEMP_REWARD = []
    sli = np.random.permutation(K)
    for i in range(K):
        TEMP_CONTEXT.append(CONTEXT[sli[i]])
        TEMP_REWARD.append(REWARD[sli[i]])
    CONTEXT = TEMP_CONTEXT
    REWARD = TEMP_REWARD

    return __context_size__, __arm_size__, CONTEXT, REWARD, sli

In [5]:
def TRANS(c, a, arm_size):
    #### transfer an array context + action to new context with dimension 2*(__context__ + __armsize__)
    dim = len(c)
    action = np.zeros(arm_size)
    action[a] = 1
    c_final = np.append(c, action)
    c_final = torch.from_numpy(c_final).to(device)
    c_final = c_final.view((len(c_final), 1))
    c_final = c_final.repeat(2, 1)
    c_final = torch.tensor(c_final, dtype=torch.float32).view(1,-1)
    return c_final

def UCB(A, phi):
    #### ucb term
    phi = phi.view(-1,1)
    try:
        tmp, LU = torch.linalg.solve(phi,A)
    except:
        A = A.detach().numpy()
        phi2 = phi.detach().numpy()
        tmp = torch.Tensor(np.linalg.solve(A, phi2))

    return torch.sqrt(torch.matmul(torch.transpose(phi,1,0), tmp))

def get_action(model, context_info, random_indicator):
    ucb = [] 
    bphi = []
    for i in range(arm_size):
        temp = TRANS(context_info, i, arm_size).to(device)
        bphi.append(temp.cpu())
        feature = model(temp).cpu()

        first_item = torch.mm(feature, theta)
        ucb_item = beta * UCB(LAMBDA, feature)
        ucb.append(first_item + ucb_item)
    
    if random_indicator == 0:
        return ucb.index(max(ucb)), bphi[ucb.index(max(ucb))] 
    else:
        num =  np.random.randint(0, arm_size)
        return num, bphi[num]
    
def update_lambda_bb(lambd, bb, context_info, action,reward):
    temp = TRANS(context_info, action, arm_size).to(device)
    feature = model(temp).cpu()
    templambda = lambd + torch.mm(feature.t(), feature)
    tempbb = bb + reward * feature.t()
    return templambda, tempbb

def update_theta(lambd, bb):
    return torch.linalg.solve(lambd, bb)

In [49]:
device = torch.device("cpu")
model = torch.load('result_folder/1thmodel.pth').to(device)
theta = np.load('result_folder/1ththeta.npy')
theta = torch.tensor(theta, device = device, dtype = torch.float32)
LAMBDA = np.load('result_folder/1thlambda.npy')
LAMBDA = torch.tensor(LAMBDA, device = device, dtype = torch.float32)
beta = 0.1

In [50]:
context_size, arm_size, CONTEXT_list, REWARD, sli = load_data("statlog", 15000)

In [51]:
predicted_label = []
for i in range(len(REWARD)):
    context = CONTEXT_list[i]
    action, c_act = get_action(model, context, 0)
    predicted_label.append(action)

C:\Users\杜骏晔\AppData\Local\Temp\ipykernel_32280\3790411606.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  c_final = torch.tensor(c_final, dtype=torch.float32).view(1,-1)


In [53]:
# get the 1 index of each array iin REWARD
label = [np.argmax(i,axis =0) for i in REWARD]

In [56]:
# Calculate the accuracy
accuracy = np.sum(np.array(predicted_label) == np.array(label))/len(label)

In [57]:
accuracy

0.9811494252873563